<a href="https://colab.research.google.com/github/mervecakir1/garbage-classifier-cnn/blob/main/garbageclassify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio plotly kaggle tqdm -q


In [ ]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(f" GPU Aktif: {len(gpus)} GPU")
else:
    print(" GPU YOK! Runtime > Change runtime type > GPU seçin")

In [ ]:
import kagglehub

path = kagglehub.dataset_download("zlatan599/garbage-dataset-classification")

print("Path to dataset files:", path)

100%|██████████| 121M/121M [00:06<00:00, 19.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zlatan599/garbage-dataset-classification/versions/5


In [ ]:
from pathlib import Path

BASE = Path("/root/.cache/kagglehub/datasets/zlatan599/garbage-dataset-classification/versions/5/Garbage_Dataset_Classification/images")

IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tif", ".tiff", ".webp"}

def count_images(folder: Path, recursive: bool = False) -> int:
    if recursive:
        return sum(1 for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in IMAGE_EXTS)
    else:
        return sum(1 for p in folder.iterdir() if p.is_file() and p.suffix.lower() in IMAGE_EXTS)

if not BASE.exists():
    raise FileNotFoundError(f"Klasör bulunamadı: {BASE}")

class_dirs = sorted([d for d in BASE.iterdir() if d.is_dir()])
total = 0

print(f"Base klasör: {BASE}\n")
for d in class_dirs:
    n = count_images(d, recursive=False)
    total += n
    print(f"{d.name:<12}: {n:5d}")

print("-" * 24)
print(f"{'TOPLAM':<12}: {total:5d}")


Base klasör: /root/.cache/kagglehub/datasets/zlatan599/garbage-dataset-classification/versions/5/Garbage_Dataset_Classification/images

cardboard   :  2214
glass       :  2500
metal       :  2084
paper       :  2315
plastic     :  2288
trash       :  2500
------------------------
TOPLAM      : 13901


In [ ]:
import os
import kagglehub

path = kagglehub.dataset_download("zlatan599/garbage-dataset-classification")
print(" Dataset path:", path)

def find_images_folder(base_path):
    """Images klasörünü recursive olarak bul"""
    for root, dirs, files in os.walk(base_path):
        class_folders = [d for d in dirs if os.path.isdir(os.path.join(root, d))]
        if len(class_folders) >= 4:
            for class_folder in class_folders[:2]:
                class_path = os.path.join(root, class_folder)
                images = [f for f in os.listdir(class_path)
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                if len(images) > 0:
                    return root
    return None

DATA_PATH = find_images_folder(path)

if DATA_PATH:
    print(f"Veri yolu bulundu: {DATA_PATH}")

    classes = [d for d in os.listdir(DATA_PATH)
              if os.path.isdir(os.path.join(DATA_PATH, d))]
    print(f" Sınıflar: {classes}")
    print(f" Sınıf sayısı: {len(classes)}")

    total_images = 0
    for class_name in classes:
        class_path = os.path.join(DATA_PATH, class_name)
        images = [f for f in os.listdir(class_path)
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"   {class_name}: {len(images)} görsel")
        total_images += len(images)

    print(f"Toplam görsel: {total_images}")
else:
    print("Veri yolu bulunamadı! Manuel kontrol edin:")
    print("Mevcut dosyalar:")
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:
            print(f"{subindent}{file}")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3, ResNet50V2, DenseNet121
from tensorflow.keras.layers import (GlobalAveragePooling2D, Dense, Dropout,
                                   BatchNormalization, Concatenate, Input, Average)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)
import warnings
warnings.filterwarnings('ignore')

class Config:
    DATA_PATH = DATA_PATH
    IMG_SIZE = (224, 224)
    BATCH_SIZE = 16
    EPOCHS = 40
    LEARNING_RATE = 1e-4
    PATIENCE = 12
    ENSEMBLE_MODEL = "ensemble_garbage_classifier.h5"
    EFFICIENTNET_MODEL = "efficientnet_garbage.h5"
    RESNET_MODEL = "resnet_garbage.h5"
    DENSENET_MODEL = "densenet_garbage.h5"
    TFLITE_MODEL = "ensemble_garbage_lite.tflite"

config = Config()

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = train_datagen.flow_from_directory(
    config.DATA_PATH,
    target_size=config.IMG_SIZE,
    batch_size=config.BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

val_data = val_datagen.flow_from_directory(
    config.DATA_PATH,
    target_size=config.IMG_SIZE,
    batch_size=config.BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

num_classes = train_data.num_classes
class_names = list(train_data.class_indices.keys())

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))

print("Class weights:")
for i, (class_name, weight) in enumerate(zip(class_names, class_weights)):
    print(f"   {class_name}: {weight:.2f}")

def create_base_model(model_name, input_shape, num_classes):
    if model_name == "efficientnet":
        base = EfficientNetB3(weights='imagenet', include_top=False, input_shape=input_shape)
        unfreeze_layers = 50
    elif model_name == "resnet":
        base = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
        unfreeze_layers = 40
    elif model_name == "densenet":
        base = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
        unfreeze_layers = 30

    for layer in base.layers:
        layer.trainable = False
    for layer in base.layers[-unfreeze_layers:]:
        layer.trainable = True

    x = base.output
    x = GlobalAveragePooling2D(name=f'{model_name}_gap')(x)
    x = BatchNormalization(name=f'{model_name}_bn1')(x)
    x = Dense(512, activation='relu', name=f'{model_name}_dense1')(x)
    x = Dropout(0.5, name=f'{model_name}_dropout1')(x)
    x = BatchNormalization(name=f'{model_name}_bn2')(x)
    x = Dense(256, activation='relu', name=f'{model_name}_dense2')(x)
    x = Dropout(0.3, name=f'{model_name}_dropout2')(x)
    output = Dense(num_classes, activation='softmax', name=f'{model_name}_output')(x)

    model = Model(inputs=base.input, outputs=output, name=f'{model_name}_model')
    return model

def create_ensemble_model(input_shape, num_classes):
    input_layer = Input(shape=input_shape, name='ensemble_input')

    efficientnet = create_base_model("efficientnet", input_shape, num_classes)
    resnet = create_base_model("resnet", input_shape, num_classes)
    densenet = create_base_model("densenet", input_shape, num_classes)

    eff_output = efficientnet(input_layer)
    resnet_output = resnet(input_layer)
    dense_output = densenet(input_layer)

    ensemble_output = Average(name='ensemble_average')([eff_output, resnet_output, dense_output])
    ensemble_model = Model(inputs=input_layer, outputs=ensemble_output, name='EnsembleGarbageClassifier')
    return ensemble_model, [efficientnet, resnet, densenet]

ensemble_model, individual_models = create_ensemble_model((*config.IMG_SIZE, 3), num_classes)

print(f" Total Parameters: {ensemble_model.count_params():,}")

ensemble_model.summary()

ensemble_model.compile(
    optimizer=AdamW(learning_rate=config.LEARNING_RATE, weight_decay=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top3_acc')]
)

callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=config.PATIENCE,
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.3,
        patience=8,
        min_lr=1e-7,
        verbose=1,
        cooldown=3
    ),
    ModelCheckpoint(
        config.ENSEMBLE_MODEL,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1,
        mode='max'
    )
]

history = ensemble_model.fit(
    train_data,
    validation_data=val_data,
    epochs=config.EPOCHS,
    class_weight=class_weights_dict,
    callbacks=callbacks,
    verbose=1
)

def plot_training_history(history):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    axes[0, 0].plot(history.history['accuracy'], 'b-', label='Training', linewidth=2)
    axes[0, 0].plot(history.history['val_accuracy'], 'r-', label='Validation', linewidth=2)
    axes[0, 0].set_title('Ensemble Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].plot(history.history['loss'], 'b-', label='Training', linewidth=2)
    axes[0, 1].plot(history.history['val_loss'], 'r-', label='Validation', linewidth=2)
    axes[0, 1].set_title('Ensemble Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    axes[1, 0].plot(history.history['top3_acc'], 'g-', label='Training Top-3', linewidth=2)
    axes[1, 0].plot(history.history['val_top3_acc'], 'orange', label='Validation Top-3', linewidth=2)
    axes[1, 0].set_title('Top-3 Accuracy', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Top-3 Accuracy')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    axes[1, 1].axis('off')
    best_val_acc = max(history.history['val_accuracy'])
    best_val_top3 = max(history.history['val_top3_acc'])
    min_val_loss = min(history.history['val_loss'])

    summary_text = f"""
     ENSEMBLE TRAINING SUMMARY
     Architecture: 3-Model Ensemble
       • EfficientNetB3
       • ResNet50V2
       • DenseNet121
     Best Metrics:
       • Val Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)
       • Val Top-3 Acc: {best_val_top3:.4f} ({best_val_top3*100:.2f}%)
       • Val Loss: {min_val_loss:.4f}
     Training Info:
       • Total Epochs: {len(history.history['accuracy'])}
       • Batch Size: {config.BATCH_SIZE}
       • Learning Rate: {config.LEARNING_RATE}
    """

    axes[1, 1].text(0.05, 0.95, summary_text, transform=axes[1, 1].transAxes,
                   fontsize=11, verticalalignment='top',
                   bbox=dict(boxstyle="round,pad=0.4", facecolor="lightblue", alpha=0.8))

    plt.tight_layout()
    plt.show()

plot_training_history(history)
print(f" En iyi model kaydedildi: {config.ENSEMBLE_MODEL}")

In [ ]:
from tensorflow import keras
model = keras.models.load_model("ensemble_garbage_classifier.h5")
print(".h5 model yüklendi")

In [ ]:
model.save("ensemble_garbage_classifier.keras")
print(".keras formatında kaydedildi")

In [ ]:
import os

h5_size = os.path.getsize("ensemble_garbage_classifier.h5") / (1024*1024)
keras_size = os.path.getsize("ensemble_garbage_classifier.keras") / (1024*1024)


In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

ensemble_model = keras.models.load_model(config.ENSEMBLE_MODEL)
print(f" En iyi model yüklendi: {config.ENSEMBLE_MODEL}")

val_data.reset()
print(f" Validation samples: {val_data.samples}")

predictions = ensemble_model.predict(val_data, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = val_data.classes

print(f" Tahmin edilen sample sayısı: {len(y_pred)}")
print(f" Gerçek sample sayısı: {len(y_true)}")

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_true, y_pred)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=class_names, yticklabels=class_names,
           square=True, cbar_kws={'shrink': 0.8})

plt.title('Confusion Matrix - Ensemble Model', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 10))
sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Greens',
           xticklabels=class_names, yticklabels=class_names,
           square=True, cbar_kws={'shrink': 0.8})
plt.title('Normalized Confusion Matrix - Ensemble Model', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("=" * 80)
report = classification_report(y_true, y_pred, target_names=class_names, digits=4)
print(report)

from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')
weighted_f1 = f1_score(y_true, y_pred, average='weighted')

print("=" * 60)
print(f" Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f" Macro F1-Score: {macro_f1:.4f}")
print(f" Weighted F1-Score: {weighted_f1:.4f}")

print("=" * 80)
print(f"{'Class':<12} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'Support':<8}")
print("-" * 80)

for i, class_name in enumerate(class_names):
    print(f"{class_name:<12} {precision[i]:<10.4f} {recall[i]:<10.4f} {f1[i]:<10.4f} {support[i]:<8}")

best_idx = np.argmax(f1)
worst_idx = np.argmin(f1)
print(f"\n Best Performance: {class_names[best_idx]} (F1: {f1[best_idx]:.4f})")
print(f" Worst Performance: {class_names[worst_idx]} (F1: {f1[worst_idx]:.4f})")

print("=" * 50)
confidence_scores = np.max(predictions, axis=1)
print(f"Average Confidence: {np.mean(confidence_scores):.4f}")
print(f"Std Confidence: {np.std(confidence_scores):.4f}")
print(f"Min Confidence: {np.min(confidence_scores):.4f}")
print(f"Max Confidence: {np.max(confidence_scores):.4f}")


low_confidence_threshold = 0.7
low_confidence_indices = np.where(confidence_scores < low_confidence_threshold)[0]
print(f"Low confidence predictions (< {low_confidence_threshold}): {len(low_confidence_indices)} / {len(confidence_scores)} ({len(low_confidence_indices)/len(confidence_scores)*100:.1f}%)")

plt.figure(figsize=(10, 6))
plt.hist(confidence_scores, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(np.mean(confidence_scores), color='red', linestyle='--',
           label=f'Mean: {np.mean(confidence_scores):.3f}')
plt.axvline(low_confidence_threshold, color='orange', linestyle='--',
           label=f'Threshold: {low_confidence_threshold}')
plt.title('Prediction Confidence Distribution - Ensemble Model', fontsize=14, fontweight='bold')
plt.xlabel('Confidence Score')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("=" * 60)

ensemble_model.save("ensemble_garbage_classifier.keras")
print(" .keras format saved")

print("Converting to TensorFlow Lite...")
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(ensemble_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = None
tflite_model = converter.convert()

with open(config.TFLITE_MODEL, 'wb') as f:
    f.write(tflite_model)

print(f" TFLite model saved: {config.TFLITE_MODEL}")

import os
h5_size = os.path.getsize(config.ENSEMBLE_MODEL) / (1024*1024)
keras_size = os.path.getsize("ensemble_garbage_classifier.keras") / (1024*1024)
tflite_size = os.path.getsize(config.TFLITE_MODEL) / (1024*1024)

print(f" H5 Format: {h5_size:.1f} MB")
print(f"Keras Format: {keras_size:.1f} MB")
print(f"TFLite Format: {tflite_size:.1f} MB")
print(f" Compression Ratio: {h5_size/tflite_size:.1f}x smaller")

print("=" * 60)

import gradio as gr
from PIL import Image
import io

def predict_garbage(image):
    try:

        if image is None:
            return " Lütfen bir görsel yükleyin!"
        img = image.resize(config.IMG_SIZE)
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        predictions = ensemble_model.predict(img_array, verbose=0)

        top_indices = np.argsort(predictions[0])[::-1][:3]

        results = {}
        for i, idx in enumerate(top_indices):
            confidence = predictions[0][idx]
            class_name = class_names[idx]
            results[f"{i+1}. {class_name}"] = f"{confidence:.3f} ({confidence*100:.1f}%)"

        return results

    except Exception as e:
        return f" Hata: {str(e)}"

def predict_with_visualization(image):
    """Görselleştirme ile prediction"""
    try:
        if image is None:
            return "Lütfen bir görsel yükleyin!", None
        img = image.resize(config.IMG_SIZE)
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        predictions = ensemble_model.predict(img_array, verbose=0)[0]

        fig, ax = plt.subplots(figsize=(10, 6))
        bars = ax.bar(class_names, predictions, color='skyblue', alpha=0.7)

        max_idx = np.argmax(predictions)
        bars[max_idx].set_color('red')
        bars[max_idx].set_alpha(0.8)

        ax.set_title(f'Ensemble Model Predictions\nTop: {class_names[max_idx]} ({predictions[max_idx]*100:.1f}%)',
                    fontsize=14, fontweight='bold')
        ax.set_ylabel('Confidence Score')
        ax.set_xlabel('Waste Categories')
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
        buf.seek(0)
        plot_img = Image.open(buf)
        plt.close()

        top_pred = class_names[max_idx]
        confidence = predictions[max_idx]

        result_text = f"""
         **ENSEMBLE MODEL PREDICTION**

        **Top Prediction:** {top_pred}
        **Confidence:** {confidence:.3f} ({confidence*100:.1f}%)

        **All Predictions:**
        """

        for i, (name, conf) in enumerate(zip(class_names, predictions)):
            result_text += f"\n{i+1}. {name}: {conf:.3f} ({conf*100:.1f}%)"

        return result_text, plot_img

    except Exception as e:
        return f" Hata: {str(e)}", None
simple_interface = gr.Interface(
    fn=predict_garbage,
    inputs=gr.Image(type="pil", label=" Çöp Fotoğrafı Yükleyin"),
    outputs=gr.Label(num_top_classes=3, label=" Tahmin Sonuçları"),
    title="AI Ensemble Çöp Sınıflandırıcı",
    description="""
    **3-Model Ensemble (EfficientNetB3 + ResNet50V2 + DenseNet121)**

    Çöp fotoğrafınızı yükleyin ve AI'ın hangi kategoriye ait olduğunu tahmin etmesini izleyin!

    **Kategoriler:** Cardboard, Glass, Metal, Paper, Plastic, Trash
    """,
    examples=None,
    allow_flagging="never"
)
advanced_interface = gr.Interface(
    fn=predict_with_visualization,
    inputs=gr.Image(type="pil", label=" Çöp Fotoğrafı"),
    outputs=[
        gr.Textbox(label="Detaylı Sonuç"),
        gr.Image(label=" Confidence Chart")
    ],
    title="Advanced Ensemble Çöp Sınıflandırıcı",
    description="""
    **Gelişmiş Analiz:** Tüm sınıflar için confidence skorları ve görselleştirme
    """,
    allow_flagging="never"
)
demo = gr.TabbedInterface(
    [simple_interface, advanced_interface],
    ["Basit Tahmin", " Detaylı Analiz"],
    title=" Ensemble AI Çöp Sınıflandırıcı"
)

print("=" * 40)

demo.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("=" * 60)
print(" Model Performance Summary:")
print(f"    Accuracy: {accuracy:.3f} ({accuracy*100:.1f}%)")
print(f"    F1-Score: {macro_f1:.3f}")
print(f"    Best Class: {class_names[best_idx]} ({f1[best_idx]:.3f})")
print(f"    Worst Class: {class_names[worst_idx]} ({f1[worst_idx]:.3f})")
print(f"    {config.ENSEMBLE_MODEL} ({h5_size:.1f} MB)")
print(f"    ensemble_garbage_classifier.keras ({keras_size:.1f} MB)")
print(f"    {config.TFLITE_MODEL} ({tflite_size:.1f} MB)")
